In [ ]:
# importing sys
import sys
 
# adding Folder_2 to the system path
sys.path.insert(0, './common/Python_Modules/')

from AFEP_parse import *
import matplotlib.pyplot as plt
from IPython.display import display, Markdown

import warnings #Suppress future warnings from pandas.
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
#os.getcwd()

# What this Notebook Does:

Each section of this notebook will calculate a component of the free energy of binding of phenol to lysozyme as described in the companion tutorial. Briefly, during a free energy perturbation simulation, NAMD (or other software) will write the difference in internal energy between the simulated state and an adjascent state (dE =  E_lambda_k+/-1 - E_lambda_k, where lambda_k determines the ensemble being simulated). The functions used by this notebook read, parse, and process those outputs into a standard format that can be quickly analyzed using one of several methods. By default, we use the Bennett Acceptance Ratio (BAR) estimator with automated decorrelation to make the calculations more robust to both outliers and autocorrelation. See Shirts and Chodera (2018) for more details.

One section of the notebook uses thermodynamic integration (TI) to calculate the free energy cost of imposing the DBC. This calculation is much more straight-forward than FEP calculations. We can analytically determine the derivative of the force with respect to lambda over a semi-continuous simulation (instead of using finite differences as in FEP). Simply averaging and accumulating those derivatives yields the free energy cost. See the NAMD User Guide for more details (https://www.ks.uiuc.edu/Research/namd/2.14/ug/node63.html)

# How to Use this Notebook:

## User Parameters:
The notebook as-is will read and process the sample outputs provided. 
To use it for your own data, be sure to update the section labeled "User Settings" below. Pay special attention to the *root* and *path* variables.

- root should be the path (relative or absolute) to the parent directory that contains (or will containt) all your data.
- temperature is the temperature **at which your simulations were run** 
- decorrelate is a flag for automatic decorrelation of samples (see Shirts and Chodera '08). Should be set to True for general use. 
- detectEQ (automatic equilibrium detection) flag is optional. If you have very poor convergence but believe your system is well-sampled, you may wish to try it.

## Layout:
The notebook is organized into five sections (separated by horizontal lines):
- Process the Bound Data [(step 2.2)](#bound_fep)
- Process the DBC TI calculation [(step 2.3)](#DBC_TI)
- Process the Unbound Data [(step 2.4)](#unbound_fep)
- Calculate the Volumetric Restraint Contribution [(step 2.5.2)](#volume)
- Calculate the Binding Free Energy [(step 2.5.3)](#total)

## File Structure of Sample Data:
```
Repository/Supp-Files
|
|----AFEP-Bound-Decoupling
|    |----output
|         | *.fepout
|
|----AFEP-Bulk
|    |----output
|         | *fepout
|
|----DBC_TI
|    |----output
|         | *.colvars.traj
|
|
```



# Other Important Notes and Credits
- This notebook is specially written for the SAFEP tutorial. 
For more up-to-date and general versions see the SAFEP github.

- This and other SAFEP notebooks make use of pyMBAR and Alchemlyb. 
For more information see Shirts and Chodera (2008), "Statistically optimal analysis of samples from multiple equilibrium states" doi: 10.1063/1.2978177



# User Settings:

In [ ]:
root = '.' # Root path to fepout files

# Used throughout
temperature = 300
RT = 0.00198720650096 * temperature # RT in kcal/mol

# Used in restraint perturbation calculations
radius = 3 # Radius of the spherical restraint used for the DBC TI calculation
DBC_wall = 1.5 # Position of the DBC upper wall

## Update these paths to point to your output files:

In [ ]:
bound_fep_path=f'{root}/Step2_alchemy_site/sample_output/' 
restraint_perturbation_path = f'{root}/Step3_restraint_perturbation/sample_output/'
bulk_fep_path=f'{root}/Step4_alchemy_bulk/sample_output/'

## Advanced settings:

In [ ]:
decorrelate = True #Flag for automatic decorrelation
detectEQ = False #Flag for automatic equilibrium detection

***
<a id='bound_fep'></a>
# Process the Bound FEP Data 
Here we process the FEP data generated by decoupling the ligand from the protein (**Step 2.2**)


In [ ]:
filename='*.fepout' #This can be a regex pattern if you have more than one fepout file
#Caution: if you have multiple fepout files, name them alphanumerically

fepoutFiles = glob(bound_fep_path+filename) #Resolve any naming regular expressions
u_nk, _ = readAndProcess(fepoutFiles, temperature, decorrelate, detectEQ) #u_nk stores the fep data
perWindow, cumulative = doEstimation(u_nk) #Run the BAR estimator on the fep data
forward, forward_error, backward, backward_error = doConvergence(u_nk) #Used later in the convergence plot

dG_site = np.round(cumulative.BAR.f.iloc[-1]*RT, 1)
error_site = np.round(cumulative.BAR.errors.iloc[-1]*RT, 1)
changeAndError = f'\u0394G<sub>site</sub> = {dG_site}\u00B1{error_site} kcal/mol'
Markdown('<font size=5>{}</font><br/>'.format(changeAndError))

In [ ]:
#Make some general plots
width = 8
height = 4
fig, axes = plotGeneral(cumulative, perWindow, RT, width=width, height=height, PDFtype='KDE')
plt.savefig(f'{bound_fep_path}/bound_generalFigures.pdf')
plt.show()

fig, convAx = plt.subplots(1,1)
convAx = convergencePlot(convAx, forward*RT, forward_error*RT, backward*RT, backward_error*RT)

fig.set_figwidth(width*3/4)
fig.set_figheight(height)

plt.savefig(f'{bound_fep_path}/bound_convergence.pdf')
plt.show()

***
<a id='DBC_TI'></a>
# Process the DBC TI calculation
Free energy cost of imposing the DBC (**Step 2.3**)

Note: The error estimate below is just the standard deviation of the samples. <br>
The BAR errors calculated above are standard errors of the mean.

In [ ]:
from TI_Calculations import *

In [ ]:
# Setup and processing of colvars data
dataTI = pd.read_csv(glob(restraint_perturbation_path+'*.colvars.traj')[0], delim_whitespace=True, names=['step','DBC', 'distance', 'E_dist', 'E_DBC'], comment='#', index_col=0)
dataTI = dataTI[dataTI.index>=1000][1:]
dataTI.index = dataTI.index-1000

nLambdas = 41
Lsched = np.linspace(1,0,nLambdas)

Dist = makeHarmonicWall(FC=200, upperWalls=radius, name='Dist_rest')
DBC = makeHarmonicWall(FC=0, targetFC=200, targetFE=6, upperWalls=DBC_wall, targetEQ=500, numSteps=300000, name='DBC_rest', schedule=Lsched)
Ls = (dataTI.index.values-1)//DBC['numSteps']
Ls[0] = 0
Ls[Ls==41] = 40 # This is a small hack in case there are extra samples for the last windowj

dataLs = np.round([DBC['schedule'][i] for i in Ls], 3)
dataTI.loc[:,'L'] = dataLs
dataTI = dataTI.iloc[1:]

TIperWindow, TIcumulative = processTI(dataTI, DBC, Lsched)

In [ ]:
dG_DBC = np.round(TIcumulative['dG'][1], 1)
error_DBC = np.round(TIcumulative['error'][1], 1)
changeAndError = f'\u0394G<sub>DBC</sub> = {dG_DBC}\u00B1{error_DBC} kcal/mol'
Markdown('<font size=5>{}</font><br/>'.format(changeAndError))

In [ ]:
''' Plot the results. '''
fig, axes = plotTI(TIcumulative, TIperWindow)
plt.savefig(f'{restraint_perturbation_path}/TI_general.pdf')
plt.show()

***
<a id='unbound_fep'></a>
# Process the Bulk FEP Data
Ligand decoupling from solvent (**Step 2.4**)

In [ ]:
filename='*.fepout' #This can be a regex pattern if you have more than one fepout file
fepoutFiles = glob(bulk_fep_path+filename) #Resolve any naming regular expressions
u_nk, _ = readAndProcess(fepoutFiles, temperature, decorrelate, detectEQ) #u_nk stores the fep data
perWindow, cumulative = doEstimation(u_nk) #Run the BAR estimator on the fep data
forward, forward_error, backward, backward_error = doConvergence(u_nk) #Used later in the convergence plot

dG_bulk = np.round(cumulative.BAR.f.iloc[-1]*RT, 1)
error_bulk = np.round(cumulative.BAR.errors.iloc[-1]*RT, 1)
changeAndError = f'\u0394G<sub>bulk</sub> = {dG_bulk}\u00B1{error_bulk} kcal/mol'
Markdown('<font size=5>{}</font><br/>'.format(changeAndError))

In [ ]:
width = 8
height = 4
fig, axes = plotGeneral(cumulative, perWindow, RT, width=width, height=height, PDFtype='KDE')
plt.savefig(f'{bulk_fep_path}bulk_generalFigures.pdf')
plt.show()

fig, convAx = plt.subplots(1,1)
convAx = convergencePlot(convAx, forward*RT, forward_error*RT, backward*RT, backward_error*RT)
convAx.set(xlabel='Fraction of Simulation Time', ylabel=r'Total $\rm\Delta G_{\lambda}$ (kcal/mol)')
fig.set_figwidth(width*3/4)
fig.set_figheight(height)
convAx.legend()
plt.savefig(f'{bulk_fep_path}bulk_convergence.pdf')
plt.show()

***
<a id='volume'></a>
# Calculate the Volumetric Restraint Contribution
**Step 2.5.2**


In [ ]:
molar = 1660 #cubic angstroms per particle in a one molar solution
dG_V = np.round(-RT*np.log(4/3*sp.pi*radius**3/molar), 1)
error_V = 0

In [ ]:
change = f'\u0394G<sub>V</sub> = {np.round(dG_V, 1)} kcal/mol'
Markdown('<font size=5>{}</font><br/>'.format(change))

***
<a id='total'></a>
# Calculate the Binding Free Energy
**Step 2.5.3 to End**

In [ ]:
def RMS(some_vector):
    return np.sqrt(np.mean(np.array(some_vector)**2))

In [ ]:
dG_binding = dG_bulk + dG_V + dG_DBC - dG_site
error_binding = np.sqrt(np.mean(np.array([error_bulk, error_V, error_site, error_DBC])**2))

In [ ]:
changeAndError = f'\u0394G<sub>binding</sub> = {np.round(dG_binding, 1)}\u00B1{np.round(error_binding, 1)} kcal/mol'
Markdown('<font size=5>{}</font><br/>'.format(changeAndError))

# Estimate the titration curve

In [ ]:
def P_bind(K, L):
    return L/(K+L)

def Kd(dG, RT):
    return np.exp(dG/RT)*1000

In [ ]:
concentrations = np.logspace(-3,2,1000) # A vector of concentrations in millimolar
K = Kd(dG_binding, RT)
plt.plot(concentrations, P_bind(K, concentrations), label='Binding Curve')
plt.fill_between(concentrations, P_bind(Kd(dG_binding-error_binding*1.96, RT), concentrations), P_bind(Kd(dG_binding+error_binding*1.96, RT), concentrations), alpha=0.25, label='95% Confidence Interval')
plt.xscale('log')
plt.xlabel('Concentration of Phenol (mM)')
plt.ylabel('Fraction of Sites Occupied')
plt.vlines(K, 0, 1, linestyles='dashed', color='black', label='Dissociation Constant')
plt.legend(loc='lower right')

plt.savefig(f'./titration_curve.pdf')
plt.show()